In [1]:
import os 
from dotenv import load_dotenv
import json

from pathlib import Path

from mistralai import Mistral


import json
import os
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Set

from dotenv import load_dotenv

# Add the project root to Python path for imports
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from bill_parser_engine.core.reference_resolver.bill_splitter import BillSplitter
from bill_parser_engine.core.reference_resolver.target_identifier import TargetArticleIdentifier
from bill_parser_engine.core.reference_resolver.models import BillChunk, TargetArticle
from bill_parser_engine.core.reference_resolver.config import MISTRAL_MODEL
from bill_parser_engine.core.reference_resolver.original_text_retriever import OriginalTextRetriever

from bill_parser_engine.core.reference_resolver.pipeline import BillProcessingPipeline

from pylegifrance import recherche_code
from pylegifrance.models.constants import CodeNom



load_dotenv('../.env.local')

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key)

print(f'Using mistral model {MISTRAL_MODEL}')

Using mistral model magistral-medium-2506


In [2]:
# Define file paths
bill_file = project_root / "data" / "legal_bill" / "full_legislative_bill.md"
output_dir = project_root / "scripts " / "output"

In [3]:
pipeline = BillProcessingPipeline(use_cache=True)

2025-06-12 17:55:33,313 - bill_parser_engine.core.reference_resolver.pipeline - INFO - BillProcessingPipeline initialized (component-level caching enabled)


In [4]:
pipeline.load_legislative_text_from_file(bill_file)

2025-06-12 17:55:33,319 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text: 30846 characters
2025-06-12 17:55:33,320 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text from file: /Users/duphan/Projects/bill-parser-engine/data/legal_bill/full_legislative_bill.md


In [5]:
#pipeline.clear_component_cache("original_text_retriever")
pipeline.clear_component_cache("text_reconstructor")

2025-06-12 17:55:33,330 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Invalidated 22 cache entries for component 'original_text_retriever'
2025-06-12 17:55:33,331 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared component cache for: original_text_retriever
2025-06-12 17:55:33,332 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared component cache for: text_reconstructor


In [6]:
chunks = pipeline.step_1_split_bill()

2025-06-12 17:55:33,335 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 1: Splitting legislative bill into chunks...
2025-06-12 17:55:33,341 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Split into 46 chunks


In [7]:
target_results = pipeline.step_2_identify_target_articles()

2025-06-12 17:55:33,346 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 2: Identifying target articles for 46 chunks...
2025-06-12 17:55:33,363 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Target identification complete: 28 unique articles identified


✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier


In [8]:
retrieval_results = pipeline.step_3_retrieve_original_texts()
retrieval_results

2025-06-12 17:55:33,368 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 3: Retrieving original text for target articles...
2025-06-12 17:55:33,370 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - → Fetching article L. 254-1 from code rural et de la pêche maritime
2025-06-12 17:55:33,485 - pylegifrance.auth - INFO - Legifrance API authentication successful.
2025-06-12 17:55:33,486 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/search
2025-06-12 17:55:33,975 - pylegifrance.client - INFO - API call to 'search' successful.
2025-06-12 17:55:33,977 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1
2025-06-12 17:55:33,978 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle
2025-06-12 17:55:34,170 - pylegifrance.client - INFO - API call to 'consult/getArticle' successful.
2025-06-12 17:55

[{'article_key': 'code rural et de la pêche maritime::L. 254-1',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1',
  'operation_type': 'MODIFY',
  'original_text': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d'achats ; 2° L'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d'entraide à titre gratuit au sens de l'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l'article L. 254-3 sur des exploitations dont la surface agricole utile est infé

In [9]:
pipeline._analyze_retrieval_results(retrieval_results)

{'total_articles': 28,
 'successful_retrievals': 27,
 'failed_retrievals': 1,
 'success_rate': 0.9642857142857143,
 'source_stats': {'pylegifrance': 22, 'insert_operation': 5},
 'text_length_stats': {'average': 1896.111111111111,
  'maximum': 10151,
  'minimum': 0,
  'total_characters': 51195},
 'failed_articles': ['code rural et de la pêche maritime::Titre V du livre II']}

In [10]:
reconstruction_results = pipeline.step_4_reconstruct_texts()


2025-06-12 17:55:48,891 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 4: Applying text reconstruction...


→ Processing new reconstruction with TextReconstructor


2025-06-12 17:55:55,531 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-12 17:55:55,537 - bill_parser_engine.core.reference_resolver.text_reconstructor - WARNING - Deleted text not found in original article. Deleted: 'prévu aux articles L. 254-6-2 et L. 254-6-3, incompatible avec celui des activités mentionnées aux 1...'


✓ Cached result for future use
→ Processing new reconstruction with TextReconstructor


2025-06-12 17:56:01,111 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-12 17:56:01,117 - bill_parser_engine.core.reference_resolver.text_reconstructor - WARNING - Deleted text not found in original article. Deleted: 'mentionnée aux 1° ou 2° du même II ou au IV du même article, mentionnée aux 1° ou 2° du II ou au IV ...'


...

✓ Cached result for future use
→ Processing new reconstruction with TextReconstructor
⏱️ TextReconstructor: Waiting 0.2s for rate limiting...


2025-06-12 17:58:24,982 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Cached result for future use
→ Processing new reconstruction with TextReconstructor


2025-06-12 17:58:28,874 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-12 17:58:28,878 - bill_parser_engine.core.reference_resolver.text_reconstructor - WARNING - Deleted text not found in original article. Deleted: 'végaux, notamment dans le cadre de la lutte biologique, sont soumises à une autorisation préalable. ...'
2025-06-12 17:58:28,882 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Text reconstruction complete: 26/46 successful reconstructions


✓ Cached result for future use


In [31]:
retrieval_results[2]

{'article_key': 'code rural et de la pêche maritime::L. 254-1-2',
 'code': 'code rural et de la pêche maritime',
 'article': 'L. 254-1-2',
 'operation_type': 'MODIFY',
 'original_text': "Un membre d'un organe de surveillance, d'administration et de direction d'une personne morale exerçant une activité mentionnée au 3° du II de l'article L. 254-1 ne peut être membre d'un de ces mêmes organes d'une personne morale exerçant une activité mentionnée aux 1° ou 2° du même II ou au IV du même article et un membre d'un organe de surveillance, d'administration et de direction d'une personne morale exerçant une activité mentionnée aux 1° ou 2° de ce II ou à ce IV de ce même article ne peut être membre d'un de ces mêmes organes d'une personne morale exerçant une activité mentionnée au 3° de ce II. Toutefois, une personne membre d'un organe de surveillance, d'administration ou de direction d'une personne morale exerçant une activité mentionnée aux 1° ou 2° du II de l'article L. 254-1 peut détenir, 

In [33]:
reconstruction_results[4]

{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° ter',
 'chunk_text_preview': "3° ter (nouveau) L'article L. 254-1-2 est ainsi modifié :\n\n\ta) Le premier alinéa est ainsi modifié :...",
 'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '3° ter'],
 'target_article': {'operation_type': 'MODIFY',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1-2',
  'confidence': 0.98,
  'raw_text': "L'article L. 254-1-2 est ainsi modifié"},
 'reconstruction_result': {'deleted_or_replaced_text': "mentionnée aux 1° ou 2° du même II ou au IV du même article, mentionnée aux 1° ou 2° de ce II ou à ce IV de ce même article, de ce II. Toutefois, une personne membre d'un organe de surveillance, d'administration ou de direction d'une personne morale exerçant une activité mentionnée aux 1° ou 2° du II de l'article L. 254-1 peut détenir, dans un établissement mentionné à l' article L. 510-1 bénéficiant d'un agrément pour les activités mentionnées au 3° du II de l'article L. 254-1, un mandat de pr

In [15]:
enriched_chunks = pipeline._enrich_chunks_with_target_articles()
enriched_chunks[0]

BillChunk(text='1° (Supprimé)', titre_text='# TITRE Iᴱᴿ', article_label='Article 1ᵉʳ', article_introductory_phrase='Le code rural et de la pêche maritime est ainsi modifié :', major_subdivision_label=None, major_subdivision_introductory_phrase=None, numbered_point_label='1°', hierarchy_path=['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '1°'], chunk_id='# TITRE Iᴱᴿ::Article 1ᵉʳ::1°', start_pos=565, end_pos=580, cross_references=[], target_article=TargetArticle(operation_type=<TargetOperationType.ABROGATE: 'ABROGATE'>, code='code rural et de la pêche maritime', article=None, confidence=0.8, raw_text='1° (Supprimé)', full_citation=None, version=None))

In [17]:
original_texts_lookup = pipeline._create_original_texts_lookup()
original_texts_lookup

{'code rural et de la pêche maritime::L. 254-1': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d'achats ; 2° L'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d'entraide à titre gratuit au sens de l'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l'article L. 254-3 sur des exploitations dont la surface agricole utile est inférieure ou égale à la surface définie en application du premier alinéa du V de l'article L. 732-39 , ou si les produits appliqués sont des 

In [19]:
for i, chunk in enumerate(enriched_chunks, 1):
    break

In [21]:
chunk

BillChunk(text='1° (Supprimé)', titre_text='# TITRE Iᴱᴿ', article_label='Article 1ᵉʳ', article_introductory_phrase='Le code rural et de la pêche maritime est ainsi modifié :', major_subdivision_label=None, major_subdivision_introductory_phrase=None, numbered_point_label='1°', hierarchy_path=['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '1°'], chunk_id='# TITRE Iᴱᴿ::Article 1ᵉʳ::1°', start_pos=565, end_pos=580, cross_references=[], target_article=TargetArticle(operation_type=<TargetOperationType.ABROGATE: 'ABROGATE'>, code='code rural et de la pêche maritime', article=None, confidence=0.8, raw_text='1° (Supprimé)', full_citation=None, version=None))

In [25]:
article_key = pipeline._build_article_key(chunk.target_article.code, chunk.target_article.article)
article_key

''

In [27]:
original_text = original_texts_lookup.get(article_key, "")
original_text

''